<a href="https://colab.research.google.com/github/mohammad-rahbari/federated-learning_visual-classification/blob/main/notebooks/Centralized_model_visual_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing DINO and installing its dependencies

In [ ]:
# @title Clon the DINO ripo
!git clone https://github.com/facebookresearch/dino.git

Cloning into 'dino'...
remote: Enumerating objects: 175, done.
remote: Total 175 (delta 0), reused 0 (delta 0), pack-reused 175 (from 1)
Receiving objects: 100% (175/175), 24.47 MiB | 10.97 MiB/s, done.
Resolving deltas: 100% (100/100), done.


In [ ]:
# @title Installing required dependencies regarding DINO
%cd dino
!pip install -r requirements.txt
!pip install timm

/content/dino/dino
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'



# preprocessing the CIFAR-100 dataset

feature size in CIFAR is 32x32 but DINO requires 224x224 in the input layer.

In first step we upscale the dataset and then we add randomization to it

In last step of transformation we normalize data usind mean value and standard division of ImageNet



In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [ ]:
transform_train = transforms.Compose({
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip,
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406),
                         std=(0.229, 0.224, 0.225))
})

transform_test = transforms.Compose({
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406),
                         std=(0.229, 0.224, 0.225))
})

In [ ]:
train_dataset = torchvision.datasets.CIFAR100(root='./data', train=True,
                                        download=True, transform=transform_train)
test_daataset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                       download=True, transform=transform_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
test_loader = DataLoader(test_daataset, batch_size=64, shuffle=False, num_workers=2)

100%|██████████| 169M/169M [00:13<00:00, 12.8MB/s]


# Loading and preparing the pretrained DINO model *(DINO-DeiT_Small)*

In [ ]:
# @title loadig the model
import torch.hub

model = torch.hub.load('facebookresearch/dino:main', 'dino_vits16')
# model = torch.hub.load('facebookresearch/dino:main', 'dino_xcit_small_12_p16')

Using cache found in /root/.cache/torch/hub/facebookresearch_dino_main
Downloading: "https://github.com/facebookresearch/xcit/zipball/main" to /root/.cache/torch/hub/main.zip
/usr/local/lib/python3.11/dist-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
/usr/local/lib/python3.11/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
Downloading: "https://dl.fbaipublicfiles.com/dino/dino_xcit_small_12_p8_pretrain/dino_xcit_small_12_p8_pretrain.pth" to /root/.cache/torch/hub/checkpoints/dino_xcit_small_12_p8_pretrain.pth
100%|██████████| 98.7M/98.7M [00:17<00:00, 6.02MB/s]


In [ ]:
import torch.nn as nn
#@title adding calssificaation Head

#Freeze all the backbone parameters so we only train on our new classification head
for param in model.parameters():
  param.requires_grad = False

if hasattr(model,"embed_dim"):
  feature_dim = model.embed_dim
else:
  feature_dim = 384
  # feature_dim  = model.embed_size

# Adding a new head to calssify CIFAR-100
model.head = nn.Linear(feature_dim,100)

#Makeing sure the head layer is trainable
for param in model.head.parameters():
  param.requires_grad = True

#Moving the model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

XCiT(
  (patch_embed): ConvPatchEmbed(
    (proj): Sequential(
      (0): Sequential(
        (0): Conv2d(3, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): SyncBatchNorm(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): GELU(approximate='none')
      (2): Sequential(
        (0): Conv2d(96, 192, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): SyncBatchNorm(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (3): GELU(approximate='none')
      (4): Sequential(
        (0): Conv2d(192, 384, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): SyncBatchNorm(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): ModuleList(
    (0-11): 12 x XCABlock(
      (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (attn): XCA(
        (qkv): Li